In [1]:
!pip install -U git+https://github.com/Adapter-Hub/adapter-transformers.git
!pip install datasets

  Cloning https://github.com/Adapter-Hub/adapter-transformers.git to /tmp/pip-req-build-4tmjlqk5
  Running command git clone -q https://github.com/Adapter-Hub/adapter-transformers.git /tmp/pip-req-build-4tmjlqk5
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 901kB 8.9MB/s 
     |████████████████████████████████| 3.3MB 34.2MB/s 
  Created wheel for adapter-transformers: filename=adapter_transformers-2.0.0-cp37-none-any.whl size=2097542 sha256=d8c73a2b979416a68c5c8201adeec77a36d6de97cd86564c1904b45156410532
  Stored in directory: /tmp/pip-ephem-wheel-cache-gpckgx7n/wheels/b0/56/c9/5bf1c51cd513412090ad751ab10fc025210176bf0a82dd8af3
Successfully built adapter-transformers
     |████████████████████████████████| 225kB 9.9MB/s 
     |████████████████████████████████| 112kB 15.9MB/s 
     |████████████████████████████████| 245kB 9.8MB/s 


In [2]:
import torch
import numpy
import matplotlib.pyplot as plt
import datasets
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import numpy as np
import argparse
import pdb

from transformers import DataCollatorForLanguageModeling
from transformers import LineByLineTextDataset
from transformers import RobertaTokenizer, RobertaForSequenceClassification
from transformers import Trainer, TrainingArguments
from transformers import RobertaConfig, RobertaModelWithHeads

print(torch.cuda.device_count())
print(torch.cuda.get_device_name(0))

1
Tesla P100-PCIE-16GB


In [3]:
args_dict = {
    "train_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/train.jsonl",
    "val_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/dev.jsonl",
    "test_dataset": "/content/drive/MyDrive/OMSCS_DL/SCIREC/data/test.jsonl",
    "model": "roberta-base",
    "output_dir": "/content/drive/MyDrive/OMSCS_DL/SCIREC/output/step_three/classify/",
}

In [4]:
# define accuracy metrics
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [5]:
# loading the data
train_data = datasets.load_dataset("json",data_files=args_dict["train_dataset"])["train"]
#not a typo: load_dataset loads this as training data
val_data = datasets.load_dataset(
                "json", data_files=args_dict["val_dataset"])["train"] 
test_data = datasets.load_dataset(
                "json", data_files=args_dict["test_dataset"])["train"] 

Using custom data configuration default-9f081ed18e6af576


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-9f081ed18e6af576/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-be8b4aa859efaf0f


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-be8b4aa859efaf0f/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


Using custom data configuration default-3571d51e27b61b4d


Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-3571d51e27b61b4d/0.0.0/83d5b3a2f62630efc6b5315f00f20209b4ad91a00ac586597caee3a4da0bef02. Subsequent calls will reuse this data.


In [6]:
#assign an integer key for each label
label_keys = {}
num_labels = 0
for label in train_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

for label in test_data['label']:
    if label not in label_keys:
        label_keys[label] = num_labels
        num_labels += 1

tokenizer = RobertaTokenizer.from_pretrained('roberta-base')

In [7]:
def tokenization(batched_text):
    tokenized_batch = tokenizer(batched_text['text'], padding=True, truncation=True)
    tokenized_batch["label"] = [label_keys[label] for label in batched_text["label"]]
    return tokenized_batch

train_data = train_data.map(tokenization, batched=True, batch_size=len(train_data))
val_data = val_data.map(tokenization, batched=True, batch_size=len(val_data))
test_data = test_data.map(tokenization, batched=True, batch_size=len(test_data))

In [8]:
train_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
val_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
test_data.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])

In [14]:
f1_scores = []
for seed in range(1,4):
  #create model
  model_prev = RobertaForSequenceClassification.from_pretrained(
        "roberta-base", hidden_dropout_prob=0.1, num_labels=num_labels)
  model_prev_configuration = model_prev.config

  model = RobertaModelWithHeads.from_pretrained(
    "roberta-base",
    config=model_prev_configuration,
  )

  model.load_adapter("/content/drive/MyDrive/OMSCS_DL/SCIREC/output/step_three/mlm/mlm_SCIREC_pretrain")

  model.add_classification_head(
  'mlm_SCIREC_pretrain',
  num_labels=num_labels,
  )
  # Activate the adapter
  model.train_adapter('mlm_SCIREC_pretrain')

  #adapter_name = args_dict["adapter_name"] + str(seed)
    


  training_args = TrainingArguments(
      num_train_epochs=20,
      per_device_train_batch_size=16,
      per_device_eval_batch_size=16,
      learning_rate=6e-5,
      lr_scheduler_type="constant",
      fp16=True,
      eval_accumulation_steps=20,
      save_strategy="no",
      save_steps=5000,
      save_total_limit=1,
      load_best_model_at_end=False,
      metric_for_best_model="f1",
      evaluation_strategy="epoch",
      output_dir=args_dict["output_dir"],
      overwrite_output_dir=True,
      # The next line is important to ensure the dataset labels are properly passed to the model
      remove_unused_columns=False,
      seed = seed
  )

  trainer = Trainer(
      model = model,
      args=training_args,
      compute_metrics=compute_metrics,
      train_dataset=train_data,
      eval_dataset=val_data
  )


  trainer.train()

  metrics = trainer.evaluate(test_data)
  print("Metrics for seed {}: {}".format(seed, metrics))

  f1_scores.append(metrics["eval_f1"])

  model.save_adapter(args_dict["output_dir"] + str(seed) + "/", "mlm_SCIREC_pretrain", with_head=True)


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.625120,0.470330,0.091394,0.067190,0.142857,4.994600,91.098000
2,No log,1.595450,0.470330,0.091394,0.067190,0.142857,4.945100,92.011000
3,1.522500,1.242929,0.582418,0.262137,0.371333,0.294221,4.777900,95.231000
4,1.522500,1.002542,0.663736,0.408516,0.510521,0.411015,5.186100,87.734000
5,1.033100,1.004801,0.663736,0.416954,0.494718,0.424608,4.937300,92.156000
6,1.033100,0.727544,0.767033,0.583047,0.679675,0.574774,4.877800,93.279000
7,1.033100,0.672342,0.782418,0.601929,0.714158,0.612808,4.905400,92.755000
8,0.683300,0.612330,0.819780,0.713987,0.783480,0.694878,4.752400,95.741000
9,0.683300,0.599017,0.810989,0.704764,0.806495,0.676267,4.814300,94.511000
10,0.504700,0.527739,0.839560,0.774188,0.806634,0.750170,4.879400,93.250000


/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Metrics for seed 1: {'eval_loss': 0.6597360372543335, 'eval_accuracy': 0.8388090349075975, 'eval_f1': 0.7630209853649278, 'eval_precision': 0.7541368028562648, 'eval_recall': 0.7804080142655737, 'eval_runtime': 10.1629, 'eval_samples_per_second': 95.839, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': -7168, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 141997056}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.651215,0.470330,0.091394,0.067190,0.142857,4.754000,95.708000
2,No log,1.623548,0.470330,0.091394,0.067190,0.142857,4.739200,96.008000
3,1.535700,1.461848,0.534066,0.173687,0.138937,0.251658,4.844300,93.925000
4,1.535700,1.112719,0.626374,0.290418,0.288894,0.351469,4.917900,92.519000
5,1.176300,0.836824,0.729670,0.511124,0.543468,0.513800,5.064100,89.849000
6,1.176300,0.684477,0.780220,0.627078,0.659709,0.619251,4.771100,95.365000
7,1.176300,0.701328,0.782418,0.642253,0.787029,0.634883,4.925600,92.374000
8,0.698100,0.609731,0.795604,0.694768,0.723370,0.691643,4.823700,94.326000
9,0.698100,0.532225,0.819780,0.740146,0.756575,0.729975,5.175600,87.913000
10,0.497700,0.520908,0.819780,0.728083,0.764091,0.740677,5.158600,88.203000


Metrics for seed 2: {'eval_loss': 0.5196585655212402, 'eval_accuracy': 0.8655030800821355, 'eval_f1': 0.7784308135951922, 'eval_precision': 0.7991042318712813, 'eval_recall': 0.7677863289919705, 'eval_runtime': 10.218, 'eval_samples_per_second': 95.322, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': 0, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 0, 'eval_mem_gpu_peaked_delta': 142002176}


Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifie

Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall,Runtime,Samples Per Second
1,No log,1.575647,0.470330,0.091394,0.067190,0.142857,4.857700,93.665000
2,No log,1.432308,0.523077,0.158988,0.132291,0.204476,4.898600,92.884000
3,1.447200,1.237496,0.600000,0.259027,0.237198,0.296185,5.070600,89.733000
4,1.447200,0.948561,0.652747,0.364304,0.463153,0.395950,4.810700,94.581000
5,1.047500,0.787822,0.745055,0.548346,0.578369,0.536945,4.858400,93.653000
6,1.047500,0.669310,0.767033,0.559184,0.662300,0.593270,4.778700,95.214000
7,1.047500,0.521635,0.817582,0.705001,0.824842,0.669315,4.948000,91.956000
8,0.660900,0.475570,0.830769,0.733099,0.824820,0.713851,4.879000,93.256000
9,0.660900,0.438667,0.848352,0.771910,0.828449,0.754660,4.940600,92.094000
10,0.461700,0.417049,0.850549,0.779729,0.832200,0.750047,4.802300,94.747000


Metrics for seed 3: {'eval_loss': 0.5626511573791504, 'eval_accuracy': 0.8767967145790554, 'eval_f1': 0.8067608308052204, 'eval_precision': 0.8165328202095358, 'eval_recall': 0.8085680907692004, 'eval_runtime': 10.2907, 'eval_samples_per_second': 94.649, 'epoch': 20.0, 'eval_mem_cpu_alloc_delta': -4096, 'eval_mem_gpu_alloc_delta': 0, 'eval_mem_cpu_peaked_delta': 4096, 'eval_mem_gpu_peaked_delta': 136128000}


In [15]:
print("Avg f1 score: {}, stdev: {}".format(np.mean(f1_scores), np.std(f1_scores)))

Avg f1 score: 0.7827375432551135, stdev: 0.018114533024764148
